In [ ]:
# Raster Data for egions is merged, creating one raster for the diff DEMs, the slopes and the aspect of the region

In [2]:
import glob
import geopandas as gpd
import rasterio
from rasterio.merge import merge
from shapely.geometry import box
import numpy as np

def merge_rasters_and_create_boundary(type, region, year_start, year_end, output_suffix, description):
    # Define paths
    paths = f"DEM_rasters/DEM Raster/{region}/*"  # Forward slashes for paths
    fileList = glob.glob(f'{paths}/{type}_*_{year_start}_*_{year_end}_*.tif')

    # Merge all rasters
    print(f'{region}')
    print(f"Merging {description}")
    src_files_to_mosaic = [rasterio.open(item) for item in fileList]

    # Merge rasters, ignoring nodata values set as np.nan
    mosaic, out_trans = merge(src_files_to_mosaic, nodata=np.nan)

    # Define and write the output raster file
    output_path = f"DEM_rasters/DEM_raster_combined/{region}/{output_suffix}_{region}_{year_start}_A_{year_end}.tif"
    out_meta = src_files_to_mosaic[0].meta.copy()
    out_meta.update({"driver": "GTiff", "height": mosaic.shape[1], "width": mosaic.shape[2],
                     "transform": out_trans, "nodata": np.nan})
    with rasterio.open(output_path, "w", **out_meta) as dest:
        dest.write(mosaic)

    # Create boundary if it's a DEM type
    if type == 'diff_dem':
        bounds = box(*rasterio.transform.array_bounds(out_meta['height'], out_meta['width'], out_meta['transform']))
        gdf = gpd.GeoDataFrame(geometry=[bounds], crs=src_files_to_mosaic[0].crs)
        gdf.to_file(f"DEM_rasters/DEM_boundries_shapes/{region}/DEM_shape_{region}_{year_start}_A_{year_end}.geojson", driver='GeoJSON')

def merge_terrain_attributes(attribute, regions):
    for region in regions:
        paths = f"terrain_attributes/{region}/*/{attribute}_*.tif"
        fileList = glob.glob(paths)
        output = f"terrain_attributes/{region}_{attribute}s.tif"
        gdal.Warp(output, fileList, format='GTiff')

# Configuration for regions and years
regions_years = {
    'peel': [('2010', '2016'), ('2011', '2016'), ('2010', '2021'), ('2011', '2021'), ('2013', '2021')],
    'kolguev': [('2010', '2017')],
    'herschel': [('2010', '2017'), ('2010', '2018'), ('2010', '2020')],
    'gydan': [('2010', '2020'), ('2010', '2021')]
}

# Process DEM and SAR for each region and year
for region, year_pairs in regions_years.items():
    for year_start, year_end in year_pairs:
        merge_rasters_and_create_boundary('diff_dem', region, year_start, year_end, 'DEM_merged', 'DEMs')
        merge_rasters_and_create_boundary('sar_mask', region, year_start, year_end, 'SAR_merged', 'SAR')

# Merge aspect and slope mosaics
merge_terrain_attributes('aspect', ['herschel', 'peel', 'kolguev', 'gydan'])
merge_terrain_attributes('slope', ['herschel', 'peel', 'kolguev', 'gydan'])


Merging DEMs
Merging SAR
Merging DEMs
Merging SAR
Merging DEMs
Merging SAR
Merging DEMs
Merging SAR
Merging DEMs
Merging SAR
Merging DEMs
Merging SAR
Merging DEMs
Merging SAR
Merging DEMs
Merging SAR
Merging DEMs
Merging SAR
Merging DEMs
Merging SAR
Merging DEMs
Merging SAR


c:\Users\Noe\anaconda3\envs\geo_env\Lib\site-packages\osgeo\gdal.py:312: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(
